In [1]:
import pandas as pd

def calc_conf(row):
    if row['label'] == 1:
        return (1-row['conf'])
    else:
        return row['conf']  

def calc_conf_1(row):
    if row['label'] == 0:
        return (1-row['conf_1'])
    else:
        return row['conf_1']

In [2]:
train_y = pd.read_csv('/Users/tinac/Downloads/NLP/jupyter/yelp_50k.csv')
train_y.drop(train_y.columns.difference(['label']), 1, inplace=True)

test_y = pd.read_csv('/Users/tinac/Downloads/NLP/jupyter/labelled_10k.csv')
test_y.drop(test_y.columns.difference(['content','label']), 1, inplace=True)

# test_y.head()
# train_y.head()
print("train",train_y.shape, "test",test_y.shape)

train (49639, 1) test (975, 2)


In [56]:
# MODEL 1: Logistic Regression

log_train = pd.read_csv('/Users/tinac/Downloads/NLP/jupyter/log_train.csv')
log_train_copy = log_train.copy()
log_train ['conf_1'] = log_train.loc[:, 'conf']
log_train["conf"] = log_train.apply(calc_conf, axis=1)
log_train["conf_1"] = log_train.apply(calc_conf_1, axis=1)
log_train.drop(log_train.columns.difference(['conf','conf_1']), 1, inplace=True)

# log_train.head()
# print(log_train.shape)

log_test = pd.read_csv('/Users/tinac/Downloads/NLP/jupyter/log_test.csv')
log_test_copy = log_test.copy()
log_test ['conf_1'] = log_test.loc[:, 'conf']
log_test["conf"] = log_test.apply(calc_conf, axis=1)
log_test["conf_1"] = log_test.apply(calc_conf_1, axis=1)
log_test.drop(log_test.columns.difference(['conf','conf_1']), 1, inplace=True)


# log_test.head()
# print(log_test.shape)

(975, 2)


In [58]:
# MODEL 2: Hugging Face (Transformer)

trans_train = pd.read_csv('/Users/tinac/Downloads/NLP/jupyter/trans_train.csv')
trans_train = trans_train.rename({'score': 'conf'}, axis=1) 
trans_train ['conf_1'] = trans_train.loc[:, 'conf']
trans_train["conf"] = trans_train.apply(calc_conf, axis=1)
trans_train["conf_1"] = trans_train.apply(calc_conf_1, axis=1)
trans_train.drop(trans_train.columns.difference(['conf','conf_1']), 1, inplace=True)

# trans_train.head()
# print(trans_train.shape)

trans_test = pd.read_csv('/Users/tinac/Downloads/NLP/jupyter/trans_test.csv')
trans_test_copy = trans_test.copy()
trans_test = trans_test.rename({'score': 'conf'}, axis=1) 
trans_test ['conf_1'] = trans_test.loc[:, 'conf']
trans_test["conf"] = trans_test.apply(calc_conf, axis=1)
trans_test["conf_1"] = trans_test.apply(calc_conf_1, axis=1)
trans_test.drop(trans_test.columns.difference(['conf','conf_1']), 1, inplace=True)

# trans_test.head()
# print(trans_test.shape)

In [61]:
# MODEL 3: NN

nn_train = pd.read_csv('/Users/tinac/Downloads/NLP/jupyter/nn_train.csv')
nn_train = nn_train.rename({'pred_label': 'label', 'confidence': 'conf'}, axis=1) 
nn_train ['conf_1'] = nn_train.loc[:, 'conf']
nn_train["conf"] = nn_train.apply(calc_conf, axis=1)
nn_train["conf_1"] = nn_train.apply(calc_conf_1, axis=1)
nn_train.drop(nn_train.columns.difference(['conf','conf_1']), 1, inplace=True)

# nn_train.head()
# print(nn_train.shape)

nn_test = pd.read_csv('/Users/tinac/Downloads/NLP/jupyter/nn_test.csv')
nn_test_copy = nn_test.copy()
nn_test = nn_test.rename({'pred_label': 'label', 'confidence': 'conf'}, axis=1) 
nn_test ['conf_1'] = nn_test.loc[:, 'conf']
nn_test["conf"] = nn_test.apply(calc_conf, axis=1)
nn_test["conf_1"] = nn_test.apply(calc_conf_1, axis=1)
nn_test.drop(nn_test.columns.difference(['conf','conf_1']), 1, inplace=True)

# nn_test.head()
# print(nn_test.shape)

In [34]:
# Combining 3 Models

df = pd.concat([log_train,trans_train, nn_train], axis=1, ignore_index=True)
df.head()

trainX = df.to_numpy()
trainY = train_y.label.values.tolist()

# print(trainX)

df2 = pd.concat([log_test,trans_test, nn_test], axis=1, ignore_index=True)

testX = df2.to_numpy()
testY = test_y.label.values.tolist()
testY = [int(x) for x in testY]
testY = numpy.array(testY)

# print(testY)

In [32]:
from sklearn.linear_model import LogisticRegression
 
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(train_x, train_y):
    model = LogisticRegression()
    print(train_x.shape)
    model.fit(train_x, train_y)
    return model

# make a prediction with the stacked model
def stacked_prediction(testX):
    yhat = model.predict(testX)
    return yhat
 
model = fit_stacked_model(trainX, trainY)

(49639, 6)


In [66]:
from sklearn.metrics import accuracy_score

# evaluate model on test set
yhat = stacked_prediction(testX)

acc = accuracy_score(testY, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

Stacked Test Accuracy: 0.892


In [67]:
logX = log_test_copy.label.values.tolist()
logX = [int(x) for x in logX]
logX = numpy.array(logX)

acc1 = accuracy_score(testY, logX)
print('Logistic Regression Accuracy: %.3f' % acc1)

transX = trans_test_copy.label.values.tolist()
transX = [int(x) for x in transX]
transX = numpy.array(transX)

acc2 = accuracy_score(testY, transX)
print('Transformer Accuracy: %.3f' % acc2)

nnX = nn_test_copy.pred_label.values.tolist()
nnX = [int(x) for x in nnX]
nnX = numpy.array(nnX)

acc3 = accuracy_score(testY, nnX)
print('Neural Network Accuracy: %.3f' % acc3)

Logistic Regression Accuracy: 0.837
Transformer Accuracy: 0.891
Neural Network Accuracy: 0.825
